# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
#Installed using pip install citipy
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [28]:
#Create URL with API
#Temperature Units
units = "imperial"
#URL
url = "http://api.openweathermap.org/data/2.5/weather?"
#query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=830d535e54a6eb5130efb5827e5bd130&units=imperial&q='

## Generate Cities List

In [29]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

625

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [77]:
# set lists for the dataframe
city_name = []
cloudy = []
dates = []
humidity = []
lats = []
lngs = []
max_temps = []
wind_speeds = []
countries = []

# set initial count quantities for organization
count_one = 0
set_one = 1

# loops for creating dataframe columns
for city in cities:
    try:
        response = requests.get(query_url + city).json()
        cloudy.append(response['clouds']['all'])
        countries.append(response['sys']['country'])
        dates.append(response['dt'])
        humidity.append(response['main']['humidity'])
        lats.append(response['coord']['lat'])
        lngs.append(response['coord']['lon'])
        max_temps.append(response['main']['temp_max'])
        wind_speeds.append(response['wind']['speed'])
        if count_one > 48:
            count_one = 1
            set_one += 1
            city_name.append(city)
        else:
            count_one += 1
            city_name.append(city)
        print(f"Processing Record {count_one} of Set {set_one} | {city}")
    except Exception:
        print("Skipping, city was not found")
print("------------------------------\nData Retrieval Complete\n------------------------------")    

Processing Record 1 of Set 1 | khatanga
Processing Record 2 of Set 1 | nelson bay
Processing Record 3 of Set 1 | ilulissat
Processing Record 4 of Set 1 | arraial do cabo
Processing Record 5 of Set 1 | mahibadhoo
Processing Record 6 of Set 1 | port alfred
Processing Record 7 of Set 1 | beyneu
Processing Record 8 of Set 1 | esperance
Processing Record 9 of Set 1 | punta arenas
Processing Record 10 of Set 1 | iracoubo
Processing Record 11 of Set 1 | sao filipe
Processing Record 12 of Set 1 | kungurtug
Skipping, city was not found
Processing Record 13 of Set 1 | bredasdorp
Processing Record 14 of Set 1 | hervey bay
Processing Record 15 of Set 1 | lompoc
Processing Record 16 of Set 1 | utete
Processing Record 17 of Set 1 | cayenne
Processing Record 18 of Set 1 | ribeira grande
Processing Record 19 of Set 1 | pisco
Processing Record 20 of Set 1 | hobart
Processing Record 21 of Set 1 | rikitea
Processing Record 22 of Set 1 | richards bay
Processing Record 23 of Set 1 | gogrial
Processing Reco

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [80]:
#Create Dictionary
weather = ({"City": city_name,
            "Cloudiness": cloudy,
            "Country": countries,
            "Date": dates,
            "Humidity": humidity,
            "Lat": lats,
            "Lng": lngs,
            "Max Temp": max_temps,
            "Wind Speed": wind_speeds
           })
#Create DF
weather_df = pd.DataFrame(weather)
#Print DF
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,khatanga,15,RU,1651939216,100,71.9667,102.5000,-3.51,5.66
1,nelson bay,99,AU,1651939216,75,-32.7167,152.1500,54.09,1.01
2,ilulissat,1,GL,1651939216,61,69.2167,-51.1000,23.76,3.00
3,arraial do cabo,17,BR,1651939218,76,-22.9661,-42.0278,73.36,16.22
4,mahibadhoo,99,MV,1651939218,76,3.7833,72.9667,85.01,12.97
...,...,...,...,...,...,...,...,...,...
567,port augusta,0,AU,1651939452,56,-32.5000,137.7667,53.15,3.78
568,bulgan,0,MN,1651939452,58,48.8125,103.5347,48.79,5.41
569,jiuquan,100,CN,1651939453,17,39.7432,98.5174,59.54,10.92
570,debre tabor,94,ET,1651939453,34,11.8500,38.0167,69.37,7.25


In [87]:
#Export DF to CSV
weather_df.to_csv("weather_df.csv")

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [88]:
#  Get the indices of cities that have humidity over 100%.
greater_humidity = weather_df.query("Humidity > 100")
greater_humidity

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression